<a href="https://colab.research.google.com/github/jackson-trader/CS462-Email-Spam-Classifier/blob/main/CS462_Email_Spam_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Spam Likely | An Email Spam Classifier**

#### By Jackson Trader, Diego Lara, Mark Pack, & Alex Bryant
#### CS 46200 - Introduction To Artificial Intelligence

##### [Dataset from Kaggle](https://www.kaggle.com/datasets/ashfakyeafi/spam-email-classification)
##### [Link to GitHub repository](https://github.com/jackson-trader/CS462-Email-Spam-Classifier)

## **Import Libraries**

In [20]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split # To split data into training and testing sets
from sklearn.feature_extraction.text import TfidfVectorizer # TF-IDF
from sklearn.linear_model import LogisticRegression # Logistic Regression Model
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

## **Load Data**

ham = legitimate email <br>
spam = spam email

In [23]:
# Loading the data from csv file on GitHub to a pandas DataFrame
raw_email_spam_data = pd.read_csv('https://raw.githubusercontent.com/jackson-trader/CS462-Email-Spam-Classifier/refs/heads/main/email_spam_data.csv') # DataFrame
raw_email_spam_data

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...
5571,ham,Rofl. Its true to its name


In [27]:
# Replace the null values with a null string, but in this case there is no null values
email_spam_data = raw_email_spam_data.where((pd.notnull(raw_email_spam_data)), '')
email_spam_data

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...
5571,ham,Rofl. Its true to its name


In [32]:
# Checking the number of rows and columns in the DataFrame
df.shape

(5573, 2)

In [35]:
# Checking the ratio between ham vs spam
# Currently it is not balanced because there are more ham than spam
# It should be almost a 1:1 ratio between the two
df['Category'].value_counts()

,count
Category,
ham,4825
spam,747
"{""mode"":""full""",1


## **Preprocess the Messages**

## **Split the Data**

In [38]:
X = df['Message'] # Independent variable
y = df['Category'] # Dependent variable (What we are predicting)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size = 0.2,
    random_state = 100
)

print('Training set size: ', X_train.shape[0])
print('Testing set size: ', X_test.shape[0])

# Scale features

Training set size:  4458
Testing set size:  1115


## **Logistic Regression**